In [3]:
print("OK!")

OK!


In [5]:
%pip install pinecone-client

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.0 kB ? eta -:--:--
   -------------------------------------- - 204.8/214.0 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 214.0/214.0 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain import PromptTemplate 
from langchain.chains import RetrievalQA 
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

c:\Users\sharm\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "7eeff629-f281-4e06-a11b-eae034db2e00"
PINECONE_API_ENV = "gcp-starter"

In [ ]:
#EXTRACT DATA FROM THE PDF 
def load_pdf(data): 
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    Loader_cls=PyPDFLoader)
    
    documents = loader.load() 

    return documents
